# 挑选长期成长的价值股

In [1]:
import pandas as pd
import numpy as np
import time

import datetime

import sys
sys.path.append("../utils/")
import token_util
pro = token_util.set_token()

*获取股票列表*

In [2]:
stock_list = pd.read_csv("../data_pulled/stock_date_delta_priceNone.csv")
stock_list.head()

,ts_code,name
0,000002.SZ,万科A
1,000005.SZ,世纪星源
2,000006.SZ,深振业A
3,000007.SZ,全新好
4,000008.SZ,神州高铁


In [3]:
stock_list.shape

(3093, 2)

### 价值股
1.ROE>10%</br>
2.流动比率>150%，速动比率>100%</br>
3.净利润逐年增长且大于10%</br>
4.公司财务四年内都符合以上三个要求

*获取数据*

In [4]:
# 获取 roe
def get_roe(ts_code, time_list):
    for i in range(len(time_list)):
                
        ret = pro.fina_indicator(ts_code=ts_code, period='20'+str(time_list[i])+'1231', fields='ts_code,end_date,roe')
        rets = pd.concat([rets, ret], ignore_index=True) if i != 0 else ret
    
    return rets.drop_duplicates()[:5].reset_index(drop=True)
    

# 获取 income
def get_income(ts_code, time_list):
    for i in range(len(time_list)):

        ret = pro.income(ts_code=ts_code, period='20'+str(time_list[i])+'1231', fields='ts_code,end_date,n_income')
        rets = pd.concat([rets, ret], ignore_index=True) if i != 0 else ret

    return rets.drop_duplicates()[:5].reset_index(drop=True)

# 获取资产负债信息
def get_balancesheet(ts_code, time_list):
    for i in range(len(time_list)):
        
        ret = pro.balancesheet(ts_code=ts_code, period='20'+str(time_list[i])+'1231', fields='ts_code,end_date,inventories,\
                                total_cur_assets, total_cur_liab')
        ret["liquidity_ratio"] = ret["total_cur_assets"]/ret["total_cur_liab"] * 100
        ret["quick_ratio"] = (ret["total_cur_assets"]-ret["inventories"])/ret["total_cur_liab"] * 100
        rets = pd.concat([rets, ret], ignore_index=True) if i != 0 else ret
            
    rets = rets.drop(labels=["inventories","total_cur_assets","total_cur_liab"], axis=1)
    
    return rets.drop_duplicates()[:5].reset_index(drop=True)

*获取处理后的数据*

In [5]:
def get_data(stock_code, time_list):
    data = get_roe(stock_code, time_list)
    ret1 = get_income(stock_code, time_list)
    ret2 = get_balancesheet(stock_code, time_list)

    data["n_income"] = ret1["n_income"]
    data["liquidity_ratio"] = ret2["liquidity_ratio"]
    data["quick_ratio"] = ret2["quick_ratio"]
    
    return data

*进行筛选*

In [6]:
candidate_list = []
time_list = [15, 16, 17, 18, 19, 20]
times = 0
starttime = datetime.datetime.now()
len_stock = len(stock_list)

while True:
    
    # 发生中断后，防止遗漏
    start_index = 0
    
    for cur_index in range(start_index, len_stock):
        
        start_index = cur_index

        try:
            data = get_data(stock_list["ts_code"][cur_index], time_list)
        except:
            time.sleep(5)
        
        times += 1
        pass_data = True

        for i in range(1, len(data)):
            roe = data.iloc[i]["roe"]
            n_income_last = data.iloc[i-1]["n_income"]
            n_income_cur = data.iloc[i]["n_income"]
            liquidity_ratio = data.iloc[i]["liquidity_ratio"]
            quick_ratio = data.iloc[i]["quick_ratio"]

            if roe > 10 and liquidity_ratio > 150 and quick_ratio > 100 and\
                n_income_cur > 0 and n_income_cur > n_income_last and\
                (n_income_cur - n_income_last) / n_income_last >= 0.1:
                continue
            else:
                pass_data = False
                break

        if pass_data is True:
            candidate_list.append(stock_list["ts_code"][cur_index])

        if cur_index != 0 and cur_index % 100 == 0:
            endtime = datetime.datetime.now()
            print("task accomplish: {} %, cost time : {} minutes".format(round((cur_index+1)/len_stock, 2), \
                                            round((endtime - starttime).seconds / 60, 2)))

    if cur_index >= len_stock - 1:
        endtime = datetime.datetime.now()
        print("finish task !! total time : {} minutes".format(round((endtime - starttime).seconds / 60, 2)))
        break

task accomplish: 0.03 %, cost time : 8.42 minutes
task accomplish: 0.06 %, cost time : 16.0 minutes
task accomplish: 0.1 %, cost time : 21.27 minutes
task accomplish: 0.13 %, cost time : 26.8 minutes
task accomplish: 0.16 %, cost time : 31.17 minutes
task accomplish: 0.19 %, cost time : 35.63 minutes
task accomplish: 0.23 %, cost time : 40.03 minutes
task accomplish: 0.26 %, cost time : 45.63 minutes
task accomplish: 0.29 %, cost time : 50.28 minutes
task accomplish: 0.32 %, cost time : 54.85 minutes
task accomplish: 0.36 %, cost time : 59.17 minutes
task accomplish: 0.39 %, cost time : 63.48 minutes
task accomplish: 0.42 %, cost time : 67.9 minutes
task accomplish: 0.45 %, cost time : 72.15 minutes
task accomplish: 0.49 %, cost time : 76.4 minutes
task accomplish: 0.52 %, cost time : 80.65 minutes
task accomplish: 0.55 %, cost time : 84.67 minutes
task accomplish: 0.58 %, cost time : 88.65 minutes
task accomplish: 0.61 %, cost time : 93.6 minutes
task accomplish: 0.65 %, cost time : 9

In [7]:
len(candidate_list)

122

In [8]:
candidate_list

['000568.SZ',
 '000596.SZ',
 '000661.SZ',
 '000813.SZ',
 '000858.SZ',
 '000963.SZ',
 '002001.SZ',
 '002025.SZ',
 '002032.SZ',
 '002127.SZ',
 '002223.SZ',
 '002262.SZ',
 '002372.SZ',
 '002376.SZ',
 '002402.SZ',
 '002415.SZ',
 '002468.SZ',
 '002587.SZ',
 '002602.SZ',
 '002614.SZ',
 '002624.SZ',
 '002626.SZ',
 '002677.SZ',
 '002777.SZ',
 '002821.SZ',
 '002833.SZ',
 '002838.SZ',
 '002853.SZ',
 '002860.SZ',
 '002871.SZ',
 '002878.SZ',
 '002882.SZ',
 '002884.SZ',
 '002901.SZ',
 '002907.SZ',
 '002937.SZ',
 '002943.SZ',
 '300015.SZ',
 '300124.SZ',
 '300144.SZ',
 '300251.SZ',
 '300271.SZ',
 '300294.SZ',
 '300320.SZ',
 '300326.SZ',
 '300365.SZ',
 '300395.SZ',
 '300396.SZ',
 '300400.SZ',
 '300448.SZ',
 '300463.SZ',
 '300481.SZ',
 '300482.SZ',
 '300546.SZ',
 '300547.SZ',
 '300572.SZ',
 '300577.SZ',
 '300595.SZ',
 '300606.SZ',
 '300607.SZ',
 '300608.SZ',
 '300609.SZ',
 '300623.SZ',
 '300628.SZ',
 '300630.SZ',
 '300632.SZ',
 '300633.SZ',
 '300634.SZ',
 '300639.SZ',
 '300642.SZ',
 '300661.SZ',
 '3006

In [9]:
stock_list[(stock_list["ts_code"] == stock_list["ts_code"][cur_index])]

,ts_code,name
3092,603999.SH,读者传媒
